In [3]:
import numpy as np
import pandas as pd
import warnings
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# warnings.filterwarnings('ignore')
monthly=['baspread', 'beta', 'betasq','chmom',  'dolvol','idiovol', 'ill', 'indmom','maxret', 'mom12m', 'mom1m', 'mom36m', 'mom6m','mvel1',  'pricedelay', 'retvol','std_dolvol', 'std_turn',  'turn', 'zerotrade']
quarterly=['aeavol','cash', 'chtx','cinvest', 'ear','ms', 'nincr', 'roaq', 'roavol', 'roeq', 'rsup', 'stdacc', 'stdcf']
annually=['absacc', 'acc',  'age', 'agr',  'bm', 'bm_ia', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chpmia',   'convind', 'currat', 'depr', 'divi', 'divo', 'dy', 'egr', 'ep', 'gma', 'grcapx', 'grltnoa', 'herf', 'hire',  'invest', 'lev', 'lgr',  'mve_ia', 'operprof', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt', 'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc',  'ps', 'quick', 'rd', 'rd_mve', 'rd_sale', 'realestate', 'roic', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sic2', 'sin', 'sp', 'tang', 'tb']
annually.remove("sic2")
characteristics=monthly+quarterly+annually
print(len(characteristics))

94


In [4]:
df_final=pd.read_csv('./guiyi_.csv')
df_final.loc[:, 'ret-rf'] = df_final.loc[:, 'ret-rf'].clip(lower=-100.0, upper=100.0) / 100
# df_final.loc[:, 'ret-rf-fwd'] = df_final.loc[:, 'ret-rf-fwd'].clip(lower=-1.0, upper=1.0)
df_final[characteristics+['ret-rf']] = df_final[characteristics+['ret-rf']] .fillna(0) # 参考machine-learning-for-trading/20_autoencoders_for_conditional_risk_factors/06_conditional_autoencoder_for_asset_pricing_model.ipynb中的方式
firm_all=df_final['permno'].unique()
print(len(firm_all))
n_characteristics=len(characteristics)
print(df_final.head())

29853
   permno      date    ret-rf  baspread      beta    betasq     chmom  \
0   10006  19570329  0.016105 -0.625355  0.190161  0.190161  0.786187   
1   10014  19570329 -0.002300  0.911069 -0.899716 -0.899716 -0.364238   
2   10022  19570329 -0.006146  0.774834 -0.123936 -0.123936  0.048250   
3   10030  19570329  0.075607 -0.006623 -0.250710 -0.250710  0.557237   
4   10057  19570329 -0.020030 -0.350993  0.345317  0.345317 -0.403974   

     dolvol   idiovol       ill  ...  salerec  secured  securedind  sgr  \
0  0.665090 -0.400189 -0.385052  ...      0.0      0.0         0.0  0.0   
1 -0.831599  0.967833  0.973510  ...      0.0      0.0         0.0  0.0   
2 -0.678335  0.065279  0.825922  ...      0.0      0.0         0.0  0.0   
3  0.491012 -0.612110 -0.708609  ...      0.0      0.0         0.0  0.0   
4 -0.080416 -0.405866  0.089877  ...      0.0      0.0         0.0  0.0   

       sic2  sin   sp  tang   tb  ret-rf-fwd  
0  0.640492  0.0  0.0   0.0  0.0     -1.0532  
1 -0.00946

In [3]:
start_date_train = 19570401
end_date_train = 19850101
start_date_val = 19850101
end_date_val = 19960101
df_train = df_final[(df_final['date'] >= start_date_train) & (df_final['date'] < end_date_train)]
df_val = df_final[(df_final['date'] >= start_date_val) & (df_final['date'] < end_date_val)]

In [4]:
pca = PCA(n_components=6)
pca.fit(df_train[characteristics])
reg = LinearRegression()
reg.fit(pca.transform(df_train[characteristics]), df_train['ret-rf'])
output = reg.predict(pca.transform(df_val[characteristics]))
r2 = 1 - ((df_val['ret-rf'].to_numpy() - output) ** 2).sum() / (df_val['ret-rf'].to_numpy() ** 2).sum()
print(r2)

0.014635268918583244


In [5]:
start_date_train = 19570401
end_date_train = 19850101
start_date_val = 19850101
end_date_val = 19960101
for n_components in range(1, 7):
    r2s_train = []
    r2s_val = []
    lengths = []
    for _, each_df in df_final.groupby(by='permno'):
        each_df_train = each_df[(each_df['date'] >= start_date_train) & (each_df['date'] < end_date_train)]
        each_df_val = each_df[(each_df['date'] >= start_date_val) & (each_df['date'] < end_date_val)]
        if (len(each_df_train) >= 300 and len(each_df_val) >= 50):
            lengths.append(len(each_df))
            pca = PCA(n_components=n_components)
            pca.fit(each_df_train[characteristics])
            reg = LinearRegression()
            reg.fit(pca.transform(each_df_train[characteristics]), each_df_train['ret-rf'])
            output = reg.predict(pca.transform(each_df_train[characteristics]))
            r2 = 1 - ((each_df_train['ret-rf'].to_numpy() - output) ** 2).sum() / (each_df_train['ret-rf'].to_numpy() ** 2).sum()
            r2s_train.append(r2)
            output = reg.predict(pca.transform(each_df_val[characteristics]))
            r2 = 1 - ((each_df_val['ret-rf'].to_numpy() - output) ** 2).sum() / (each_df_val['ret-rf'].to_numpy() ** 2).sum()
            r2s_val.append(r2)
    print(len(lengths))
    # print(sum(1 for i in lengths if i >= 200))
    print(sum(r2s_train) / len(r2s_train))
    print(-sum(r2s_val) / len(r2s_val))

379
0.010300093975828447
-0.011384233987142953
379
0.016905569842239815
-0.010360432664519702
379
0.022025013354193365
-0.009470478707497187
379
0.02756340771503632
-0.009925160162285613
379
0.034101646111046074
-0.010410043313553703
379
0.0413307808460892
-0.009410362238571179


In [6]:
start_date_trains = [19570401, 19710101, 19810101, 19910101, 20010101]
end_date_trains = [19690101, 19790101, 19890101, 19990101, 20140101]
start_date_vals = [19690101, 19790101, 19890101, 19990101, 20140101]
end_date_vals = [19710101, 19810101, 19910101, 20010101, 20170101]
for n_components in range(1, 7):
    r2s_train = []
    r2s_val = []
    lengths = []
    for _, each_df in df_final.groupby(by='permno'):
        each_df_trains = []
        each_df_vals = []
        for i in range(len(start_date_trains)):
            each_df_trains.append(each_df[(each_df['date'] >= start_date_trains[i]) & (each_df['date'] < end_date_trains[i])])
            each_df_vals.append(each_df[(each_df['date'] >= start_date_vals[i]) & (each_df['date'] < end_date_vals[i])])
        # print(len(each_df_trains))
        # print(len(each_df_vals))
        each_df_train = pd.concat(each_df_trains)
        each_df_val = pd.concat(each_df_vals)
        if (all(len(each_df_train_decade) >= 50 for each_df_train_decade in each_df_trains) and 
            all(len(each_df_val_decade) >= 10 for each_df_val_decade in each_df_vals)):
            # print([len(each_df_train_decade) for each_df_train_decade in each_df_trains])
            # print([len(each_df_val_decade) for each_df_val_decade in each_df_vals])
            lengths.append(len(each_df))
            pca_first_step = PCA(n_components=50)
            pca_first_step.fit(each_df_train[characteristics])
            pcas_second_step = []
            regs = []
            r2_numerator_train = 0; r2_denominator_train = 0; r2_numerator_val = 0; r2_denominator_val = 0
            for each_df_train_decade, each_df_val_decade in zip(each_df_trains, each_df_vals):
                pca_second_step = PCA(n_components=n_components)
                pcas_second_step.append(pca_second_step)
                pca_second_step.fit(pca_first_step.transform(each_df_train_decade[characteristics]))
                reg = LinearRegression()
                regs.append(reg)
                reg.fit(pca_second_step.transform(pca_first_step.transform(each_df_train_decade[characteristics])), each_df_train_decade['ret-rf'])
                output = reg.predict(pca_second_step.transform(pca_first_step.transform(each_df_train_decade[characteristics])))
                r2_numerator_train += ((each_df_train_decade['ret-rf'].to_numpy() - output) ** 2).sum()
                r2_denominator_train += (each_df_train_decade['ret-rf'].to_numpy() ** 2).sum()
                output = reg.predict(pca_second_step.transform(pca_first_step.transform(each_df_val_decade[characteristics])))
                r2_numerator_val += ((each_df_val_decade['ret-rf'].to_numpy() - output) ** 2).sum()
                r2_denominator_val += (each_df_val_decade['ret-rf'].to_numpy() ** 2).sum()
            r2 = 1 - r2_numerator_train / r2_denominator_train
            r2s_train.append(r2)
            r2 = 1 - r2_numerator_val / r2_denominator_val
            r2s_val.append(r2)
    print(len(lengths))
    # print(sum(1 for i in lengths if i >= 200))
    print(sum(r2s_train) / len(r2s_train))
    print(abs(sum(r2s_val) / len(r2s_val)))

227
0.030912107845758013
0.012917799350836845
227
0.0446946457369472
0.014832316410038298
227
0.05847879716714108
0.013892286578614958
227
0.07597640583682486
0.011102181479600194
227
0.09529304632757526
0.005758521687015253
227
0.11377401806043912
0.00010807199820197589


In [7]:
start_date_train = 19570401
end_date_train = 19850101
start_date_val = 19850101
end_date_val = 19960101
for n_components in range(1, 7):
    r2s_train = []
    r2s_test = []
    lengths = []
    for _, each_df in df_final[(df_final['date'] >= start_date_train) & (df_final['date'] < end_date_train)].groupby(by='date'):
        # each_df_train = each_df[(each_df['date'] >= start_date_train) & (each_df['date'] < end_date_train)]
        # each_df_val = each_df[(each_df['date'] >= start_date_val) & (each_df['date'] < end_date_val)]
        if (len(each_df) >= 300):
            lengths.append(len(each_df))
            try:
                X_factor = np.linalg.solve(each_df[characteristics].values.T @ each_df[characteristics].values, each_df[characteristics].values.T @ each_df['ret-rf'].values)
            except np.linalg.LinAlgError:
                continue
            # X_factor=np.linalg.pinv(Z_t_minus_1.transpose(0,2,1) @ Z_t_minus_1) @ Z_t_minus_1.transpose(0,2,1) @ X_factor_ret
            input = each_df[characteristics].values.T
            pca = PCA(n_components=n_components)
            pca.fit(input[-len(input) // 10:])
            reg = LinearRegression()
            reg.fit(pca.transform(input[-len(input) // 10:]), X_factor[-len(input) // 10:])
            output = reg.predict(pca.transform(input[-len(input) // 10:]))
            r2 = 1 - ((X_factor[-len(input) // 10:] - output) ** 2).sum() / (X_factor[-len(input) // 10:] ** 2).sum()
            r2s_train.append(r2)
            output = reg.predict(pca.transform(input[:-len(input) // 10]))
            r2 = 1 - ((X_factor[:-len(input) // 10] - output) ** 2).sum() / (X_factor[:-len(input) // 10] ** 2).sum()
            r2s_val.append(r2)
    print(len(lengths))
    # print(sum(1 for i in lengths if i >= 200))
    print(sum(r2s_train) / len(r2s_train))
    print(sum(r2s_val) / len(r2s_val))

333
0.3491435387395421
-0.0020057562132578827
333
0.3725374774140128
-0.00384424893758962
333
0.4831698623636371
-0.005481820157482538
333
0.5162592611536612
-0.006866308539007843
333
0.5262077544600667
-0.008021760304960011
333
0.566404491861606
-0.00899344748599007


In [3]:
from ipca import InstrumentedPCA
# df_alternate = df_final[(df_final['date'] >= start_date_val) & (df_final['date'] < end_date_val)]\
#     .set_index(['permno', 'date']).stack().unstack(level=0).swaplevel(0, 1) # .set_index('date', append=True, inplace=True)
# print(df_alternate.head())
# df_alternate = df_final.groupby('permno').filter(lambda x : len(x) >= 200).set_index(['permno', 'date'])
# regr = InstrumentedPCA(n_factors=1, intercept=False)
# regr = regr.fit(X=df_alternate[characteristics], y=df_alternate['ret-rf'])

In [5]:
start_date_trains = [20010101, 20110101]
end_date_trains = [20100101, 20140101]
start_date_vals = [20100101, 20140101]
end_date_vals = [20110101, 20160101]
df_alternate = df_final[(df_final['date'] >= start_date_trains[0]) & (df_final['date'] < end_date_vals[-1])]\
    .set_index(['permno', 'date'])[characteristics].stack().unstack(level=0)

In [6]:
df_alternate['X_factor'] = 0
X_factors = []
df_final.fillna(0)
for date, each_df in df_final[(df_final['date'] >= start_date_trains[0]) & (df_final['date'] < end_date_vals[-1])].groupby(by='date'):
    try:
        # X_factor = np.linalg.solve(each_df[characteristics].values.T @ each_df[characteristics].values, each_df[characteristics].values.T @ each_df['ret-rf'].values)
        X_factor = np.linalg.pinv(each_df[characteristics].values.T @ each_df[characteristics].values) @ each_df[characteristics].values.T @ each_df['ret-rf'].values
        # print(X_factor)
    except np.linalg.LinAlgError:
        continue
    s = pd.Series(characteristics, X_factor)
    s['date'] = date
    df_alternate.loc[:, 'X_factor'] = pd.Series(X_factor, pd.MultiIndex.from_tuples([(date, a) for a in characteristics]))
    X_factors.append(pd.Series(X_factor, pd.MultiIndex.from_tuples([(date, a) for a in characteristics])))
    # print(X_factors[-1])
df_alternate['X_factor'] = pd.concat(X_factors)
df_alternate = df_alternate.reset_index().fillna(0)
# df_alternate = df_alternate.swaplevel(0, 1).fillna(0)
# print(df_alternate.head())

In [6]:
# permnos = df_alternate.columns.to_list()
# permnos.remove('X_factor')
# regr = InstrumentedPCA(n_factors=6, intercept=False)
# regr = regr.fit(X=df_alternate[permnos], y=df_alternate['X_factor'])

The panel dimensions are:
n_samples: 94 , L: 13674 , T: 180


: 

: 

In [7]:
permnos = df_alternate.columns.to_list()
permnos.remove('X_factor')
permnos.remove('date')
permnos.remove('level_1')
for n_components in range(1, 7):
    r2s_train = []
    r2s_val = []
    lengths = []
    for _, each_df in df_alternate.groupby(by='level_1'):
        each_df_trains = []
        each_df_vals = []
        for i in range(len(start_date_trains)):
            each_df_trains.append(each_df[(each_df['date'] >= start_date_trains[i]) & (each_df['date'] < end_date_trains[i])])
            each_df_vals.append(each_df[(each_df['date'] >= start_date_vals[i]) & (each_df['date'] < end_date_vals[i])])
        # print(len(each_df_trains))
        # print(len(each_df_vals))
        # print(each_df_trains[0])
        each_df_train = pd.concat(each_df_trains)
        each_df_val = pd.concat(each_df_vals)
        # if (all(each_df_train_decade.shape[0] * each_df_train_decade.shape[1] >= 2 * (each_df_train_decade == 0).to_numpy().sum() for each_df_train_decade in each_df_trains) and 
        #     all(each_df_val_decade.shape[0] * each_df_val_decade.shape[1] >= 2 * (each_df_val_decade == 0).to_numpy().sum() for each_df_val_decade in each_df_vals)):
        if (all(len(each_df_train_decade) >= 10 for each_df_train_decade in each_df_trains) and 
            all(len(each_df_val_decade) >= 2 for each_df_val_decade in each_df_vals)):
            # print([len(each_df_train_decade) for each_df_train_decade in each_df_trains])
            # print([len(each_df_val_decade) for each_df_val_decade in each_df_vals])
            lengths.append(len(each_df))
            pca_first_step = PCA(n_components=60)
            pca_first_step.fit(each_df_train[permnos])
            pcas_second_step = []
            regs = []
            r2_numerator_train = 0; r2_denominator_train = 0; r2_numerator_val = 0; r2_denominator_val = 0
            for each_df_train_decade, each_df_val_decade in zip(each_df_trains, each_df_vals):
                pca_second_step = PCA(n_components=n_components)
                pcas_second_step.append(pca_second_step)
                pca_second_step.fit(pca_first_step.transform(each_df_train_decade[permnos]))
                reg = LinearRegression()
                regs.append(reg)
                reg.fit(pca_second_step.transform(pca_first_step.transform(each_df_train_decade[permnos])), each_df_train_decade['X_factor'])
                output = reg.predict(pca_second_step.transform(pca_first_step.transform(each_df_train_decade[permnos])))
                # print(output)
                r2_numerator_train += ((each_df_train_decade['X_factor'].to_numpy() - output) ** 2).sum()
                r2_denominator_train += (each_df_train_decade['X_factor'].to_numpy() ** 2).sum()
                output = reg.predict(pca_second_step.transform(pca_first_step.transform(each_df_val_decade[permnos])))
                r2_numerator_val += ((each_df_val_decade['X_factor'].to_numpy() - output) ** 2).sum()
                r2_denominator_val += (each_df_val_decade['X_factor'].to_numpy() ** 2).sum()
            r2 = 1 - r2_numerator_train / r2_denominator_train
            r2s_train.append(r2)
            r2 = 1 - r2_numerator_val / r2_denominator_val
            r2s_val.append(r2)
    print(len(lengths))
    # print(sum(1 for i in lengths if i >= 200))
    print(sum(r2s_train) / len(r2s_train))
    print(abs(sum(r2s_val) / len(r2s_val)))

94
0.1520448610522434
0.06135071368849443
94
0.17043763310705068
0.03760971393058592
94
0.18519033020785103
0.031158194187123317
94
0.19784231860787102
0.006672450015151244
94
0.2110863828242271
0.01035303530909631
94
0.2302168047621838
0.03068683741123433
